# Collecte de données


In [2]:
import os
import pandas as pd

# === Récupération du type de fichier image associé à chaque pokémon ===
lst_files = os.listdir('images/images')
lst_files = [f.split('.') for f in lst_files]
dico_files = {paire[0]: paire[1] for paire in lst_files}

#importation des données depuis un fichier csv
with open('images/pokemon.csv', 'r') as csv_file:

    # === Pré-traitement des données ===
    lines = csv_file.readlines()
    lines = lines[1:] #suppression de la première ligne (entête)
    
    for i,data in enumerate(lines):
        data = data.replace('\n', '')
        lines[i] = data.split(',')
        if len(lines[i]) == 2: #si le pokémon n'a qu'un seul type
            lines[i].append(None) #ajout d'une colonne vide pour le second type
        #récupération du type de fichier image associé   
        lines[i].append(dico_files[lines[i][0]])
        
    # === Création d'un DataFrame ===      
    df = pd.DataFrame(lines, columns=["name", "type1", "type2","img_type"])
    
    csv_file.close()

# Export du DataFrame vers un fichier JSON
df.to_json('donnees.json',orient='index')

# Étiquetage et annotation

In [3]:
from PIL import Image
import numpy
import math
import pandas as pd
from sklearn.cluster import KMeans

import Color as color

# /!\ Attention, cette cellule peut prendre plusieurs minutes à s'exécuter

# === chargement des données depuis le fichier JSON ===
df = pd.read_json('donnees.json', orient='index')
#ajout d'une colonne pour les couleurs dominantes
df['colors'] = None

# === Traitement des données ===
for pokemon in df.iterrows():
    img = Image.open(f"images/images/{pokemon[1]['name']}.{pokemon[1]['img_type']}")
    numarray = numpy.array(img, numpy.uint8)
    numarray = numarray.reshape(-1, 9)
    
    # récupération des couleurs dominantes
    n_clusters = 5
    clusters = KMeans(n_clusters=n_clusters, n_init=20)
    clusters.fit(numarray)
    
    couleurs = []
    for i in range(len(clusters.cluster_centers_)):
        couleurs.append([numpy.sum(clusters.labels_==i),"#%02x%02x%02x" % (
            math.ceil(clusters.cluster_centers_[i][0]),
            math.ceil(clusters.cluster_centers_[i][1]),
            math.ceil(clusters.cluster_centers_[i][2]),
        )])
    
    #filtrage des couleurs de fond (blanc pour jpg, noir pour png)
    couleurs = sorted(couleurs, key=lambda x: x[0], reverse=True)
    if (pokemon[1]['img_type'] == 'jpg') and (color.is_near(couleurs[0][1], '#ffffff', 5, mode='HEX')):
        couleurs.pop(0)
    elif (pokemon[1]['img_type'] == 'png') and (color.is_near(couleurs[0][1], '#000000', 5, mode='HEX')):
        couleurs.pop(0)
    else:
        couleurs.pop(-1)
        
    #Normalisation des valeurs de quantité de couleurs
    sum = 0
    for paire in couleurs:
        sum += paire[0]
    for paire in couleurs:
        paire[0] = int((paire[0] / sum)* 100)
    
    # === Ajout des couleurs dominantes au DataFrame ===
    df.at[pokemon[0], 'colors'] = couleurs


# Export du DataFrame vers le fichier JSON
df.to_json('donnees.json',orient='index')

# Analyses de données

In [15]:
import ipywidgets as widgets
from PIL import Image
import os
import json
import random
from IPython.display import display

bouton_jaime = None
bouton_jaime_pas = None

def afficher_image_aleatoire(b):
    global bouton_jaime, bouton_jaime_pas

    with open("donnees.json", "r") as f:
        donnees = json.load(f)

    image_id = str(random.randint(0, len(donnees) - 1))
    nom_image = donnees[image_id]["name"]
    img_type = donnees[image_id]["img_type"]

    chemin_image = os.path.join("images/images", f"{nom_image}.{img_type}")
    image = Image.open(chemin_image)

    image_widget.value = open(chemin_image, "rb").read()

    label_nom_image.value = nom_image

    # Supprimer les anciens boutons s'ils existent
    if bouton_jaime:
        bouton_jaime.close()
    if bouton_jaime_pas:
        bouton_jaime_pas.close()

    bouton_jaime = widgets.Button(description="J'aime")
    bouton_jaime.on_click(lambda b: action_jaime(image_id))
    bouton_jaime_pas = widgets.Button(description="Je n'aime pas")
    bouton_jaime_pas.on_click(lambda b: action_jaime_pas(image_id))

    display(widgets.HBox([bouton_jaime, bouton_jaime_pas]))

def remplacement_bouton(b):
    bouton_afficher_image.close()
    afficher_image_aleatoire(b)

def action_jaime(image_id):
    modifier_preferences(image_id, 1)
    afficher_image_aleatoire(None)

def action_jaime_pas(image_id):
    modifier_preferences(image_id, -0.5)
    afficher_image_aleatoire(None)

def modifier_preferences(image_id, valeur):
    with open("donnees.json", "r+") as f:
        donnees = json.load(f)
        donnees[str(image_id)]["preferences"] = valeur
        f.seek(0)
        json.dump(donnees, f, indent=4)
        f.truncate()

image_widget = widgets.Image(format='png')
display(image_widget)

label_nom_image = widgets.Label(value="")
display(label_nom_image)

bouton_afficher_image = widgets.Button(description="Afficher une image")
bouton_afficher_image.on_click(remplacement_bouton)
display(bouton_afficher_image)


# Visualisation des données

In [ ]:
pass

# Système de recommandation

In [ ]:
pass

# Tests

pass